## KQL Query Harvester from multiple data sources

In [38]:
import logging
from pathlib import Path
import requests
import io
import zipfile
from requests.exceptions import HTTPError
import glob
import pandas as pd
import yaml
from pandas import json_normalize
import urllib.parse
import json
from pprint import pprint

def read_config(filename):
    with open(filename, "r") as yamlfile:
        data = yaml.load(yamlfile, Loader=yaml.FullLoader)
    
    return data

def get_repo_urls(repo_name, branch_name):
    git_repo_url = f'https://github.com/{repo_name}/archive/{branch_name}.zip'
    
    return git_repo_url

def download_git_archive(git_url, output_dir):
    print(f"Downloading {git_url}, may take few mins..")
    try:
        r = requests.get(git_url)
        repo_zip = io.BytesIO(r.content)
        archive = zipfile.ZipFile(repo_zip, mode="r")
        for file in archive.namelist():
            archive.extract(file, path=output_dir)
        print("Downloaded and Extracted Files successfully")
    except HTTPError as http_err:
        warnings.warn(f"HTTP error occurred trying to download from Github: {http_err}")
        
def get_sentinel_queries_from_github(git_url, outputdir):
    print("Downloading from Azure Sentinel Github, may take 2-3 mins..")
    try:
        r = requests.get(git_url)
        repo_zip = io.BytesIO(r.content)
        archive = zipfile.ZipFile(repo_zip, mode="r")
        # Only extract Detections and Hunting Queries Folder
        for file in archive.namelist():
            if (file.startswith(("Azure-Sentinel-master/Detections/","Azure-Sentinel-master/Hunting Queries/","Azure-Sentinel-master/Solutions/")) and file.endswith(".yaml")):
                archive.extract(file, path=outputdir)
        print("Downloaded and Extracted Files successfully")
    except HTTPError as http_err:
        warnings.warn(f"HTTP error occurred trying to download from Github: {http_err}")
        
def parse_yaml(parent_dir, child_dir):

    sentinel_repourl = "https://github.com/Azure/Azure-Sentinel/blob/master"
    bad_yamls = ['/home/jovyan/work/Hackathon/kql-query-store/dev-notebooks/Azure-Sentinel-master/Hunting Queries/Microsoft 365 Defender/Device Inventory/Find Software By Name and Version.yaml']
    # Collect list of files recusrively under a folder
    yaml_queries = glob.glob(f"{parent_dir}/{child_dir}/**/*.yaml", recursive=True)
    
    yaml_queries = [query for query in yaml_queries if query not in bad_yamls]
    
    df = pd.DataFrame()

    # Recursively load yaml Files and append to dataframe
    for query in yaml_queries:
        with open(query, "r", encoding="utf-8", errors="ignore") as f:
            parsed_yaml_df = json_normalize(yaml.load(f, Loader=yaml.FullLoader))
            parsed_yaml_df["GithubURL"] = urllib.parse.quote(query.replace(parent_dir, sentinel_repourl), safe=':/')
            # #URL encode
            # parsed_yaml_df["GithubURL"] = urllib.parse.quote(parsed_yaml_df["GithubURL"], safe=':/')
            # parsed_yaml_df = parsed_yaml_df[columns]
            frames = [df, parsed_yaml_df]
            df = pd.concat(frames, ignore_index=True, sort=True)

    return df

def parse_kql_to_dict(repo_name, branch_name, src_path):
    parent_dir = src_path + f"/{repo_name.split('/')[-1]}-{branch_name}"
    kql_files = glob.glob(f"{parent_dir}/**/*.kql", recursive=True)
    
    git_repo_url = f'https://github.com/{repo_name}/tree/main'

    list_of_kql_files_dict = []
    for file in kql_files:
        with open(file, "r", encoding="utf-8", errors="ignore") as f:
            file_path = Path(file)
            content = json.dumps(f.read())

            kql_files_dict = dict()
            kql_files_dict['query'] = content
            kql_files_dict['source_path'] = urllib.parse.quote(str(file_path).replace(parent_dir, git_repo_url), safe=':/')
            kql_files_dict['query_name'] = file_path.stem
            kql_files_dict['attributes'] = {}
            list_of_kql_files_dict.append(kql_files_dict)
    
    return list_of_kql_files_dict

def parse_markdown_to_dict(repo_name, branch_name, src_path):
    parent_dir = src_path + f"/{repo_name.split('/')[-1]}-{branch_name}"
    md_files = glob.glob(f"{parent_dir}/**/*.md", recursive=True)
    print(f"Processing {len(md_files)} markdonw files from repo: {repo_name}")
    git_repo_url = f'https://github.com/{repo_name}/tree/main'

    df = pd.DataFrame(columns=['query_name', 'query', 'source_path'])
    src_path_list = []
    for file in md_files:
        with open(file, "r", encoding="utf-8", errors="ignore") as f:
            file_path = Path(file)
            lines = Path(file).read_text(encoding="utf-8").split('\n')

            ct = 0
            kql = False
            kql_collect = []
            title_collect = []
            cur_kql = []
            title = "n/a"
            while ct < len(lines):
                if kql:
                    cur_kql.append(lines[ct])
                if (lines[ct].startswith("#") and lines[ct+2] == "```kql"):
                    kql = True
                    title = lines[ct]
                elif  (lines[ct] == "```kql"):
                    kql = True
                elif lines[ct] == "```":
                    kql = False
                    cur_kql = "\n".join(cur_kql)
                    kql_collect.append(cur_kql)
                    title_collect.append(title)
                    title = "n/a"
                    cur_kql = []
                ct+=1
                src_path = urllib.parse.quote(str(file_path).replace(parent_dir, git_repo_url), safe=':/')
                src_path_list.append(src_path)
            test_df = pd.DataFrame(list(zip(title_collect, kql_collect, src_path_list)), columns=['query_name', 'query', 'source_path'])

            df = pd.concat([df, test_df])
            
    return df.to_dict(orient="records")

### YAML - Azure Sentinel Githubs

In [2]:
output_dir = Path.cwd()
# download sentinel github and extract yaml files only
azsentinel_git_url = "https://github.com/Azure/Azure-Sentinel/archive/master.zip"
get_sentinel_queries_from_github(git_url=azsentinel_git_url, outputdir=output_dir)

Downloaded and Extracted Files successfully


In [5]:
# Parsing yaml files and converting to dataframe
tmp_path = str(Path.cwd())

base_dir = tmp_path + "/Azure-Sentinel-master"
detections_df = parse_yaml(parent_dir=base_dir, child_dir="Detections")
hunting_df = parse_yaml(parent_dir=base_dir, child_dir="Hunting Queries")
solutions_df = parse_yaml(parent_dir=base_dir, child_dir="Solutions")

print(f" Detections: {len(detections_df)} Hunting Queries: {len(hunting_df)} Solutions: {len(solutions_df)}")

 # Filtering yamls with no KQL queries
detections_df = detections_df[detections_df["query"].notnull()]
hunting_df = hunting_df[hunting_df["query"].notnull()]
solutions_df = solutions_df[solutions_df["query"].notnull()]

print(f" Post Filtering with null queries:  Detections: {len(detections_df)} Hunting Queries: {len(hunting_df)} Solutions: {len(solutions_df)}")

 Detections: 434 Hunting Queries: 822 Solutions: 1177
 Post Filtering with null queries:  Detections: 363 Hunting Queries: 789 Solutions: 1174


In [7]:
# Selecting specific columns
columns = ['name', 'GithubURL', 'query', 'description', 'tactics', 'relevantTechniques']
all_dfs = [detections_df[columns], hunting_df[columns], solutions_df[columns]]
sentinel_github = pd.concat(all_dfs, ignore_index=True, sort=True)

#renaming to columns to match with schema
sentinel_github.rename(columns={'GithubURL': 'source_path', 'name': 'query_name', 'relevantTechniques': 'techniques'}, inplace=True)

sentinel_github.head()

,source_path,description,query_name,query,techniques,tactics
0,https://github.com/Azure/Azure-Sentinel/blob/m...,'Anomaly Rules generate events in the Anomalie...,Unusual Anomaly,// You can leave out Anomalies that are alread...,NaN,[]
1,https://github.com/Azure/Azure-Sentinel/blob/m...,'Identifies evidence of brute force activity a...,Brute force attack against user credentials (U...,let failureCountThreshold = 10;\nlet successCo...,[T1110],[CredentialAccess]
2,https://github.com/Azure/Azure-Sentinel/blob/m...,'This query searches for failed attempts to lo...,Potential Password Spray Attack (Uses Authenti...,let FailureThreshold = 15;\nimAuthentication\n...,[T1110],[CredentialAccess]
3,https://github.com/Azure/Azure-Sentinel/blob/m...,'This query searches for successful user login...,User login from different countries within 3 h...,let timeframe = ago(3h);\nlet threshold = 2;\n...,[T1078],[InitialAccess]
4,https://github.com/Azure/Azure-Sentinel/blob/m...,'Identifies IPs with failed attempts to sign i...,Sign-ins from IPs that attempt sign-ins to dis...,imAuthentication\n| where EventResult =='Failu...,"[T1078, T1098]","[InitialAccess, Persistence]"


In [8]:
cols = ["description", "techniques", "tactics"]
# create new column by merging selected columns into dictionary
sentinel_github["attributes"] = sentinel_github[cols].to_dict(orient="records")

# select columns and display sample dataframe records
select_columns = ['source_path', 'query_name', 'query', 'attributes']
sentinel_github[select_columns].head()

,source_path,query_name,query,attributes
0,https://github.com/Azure/Azure-Sentinel/blob/m...,Unusual Anomaly,// You can leave out Anomalies that are alread...,{'description': ''Anomaly Rules generate event...
1,https://github.com/Azure/Azure-Sentinel/blob/m...,Brute force attack against user credentials (U...,let failureCountThreshold = 10;\nlet successCo...,{'description': ''Identifies evidence of brute...
2,https://github.com/Azure/Azure-Sentinel/blob/m...,Potential Password Spray Attack (Uses Authenti...,let FailureThreshold = 15;\nimAuthentication\n...,{'description': ''This query searches for fail...
3,https://github.com/Azure/Azure-Sentinel/blob/m...,User login from different countries within 3 h...,let timeframe = ago(3h);\nlet threshold = 2;\n...,{'description': ''This query searches for succ...
4,https://github.com/Azure/Azure-Sentinel/blob/m...,Sign-ins from IPs that attempt sign-ins to dis...,imAuthentication\n| where EventResult =='Failu...,{'description': ''Identifies IPs with failed a...


In [26]:
# display it as list of dictionary
sentinel_github_dict = sentinel_github[select_columns].to_dict(orient="records")
pprint(sentinel_github_dict[0])

{'attributes': {'description': "'Anomaly Rules generate events in the "
                               'Anomalies table. This scheduled rule tries to '
                               'detect Anomalies that are not usual, they '
                               'could be a type of Anomaly that has recently '
                               'been activated, or an infrequent type. The '
                               'detected Anomaly should be reviewed, if it is '
                               'relevant enough, eventually a separate '
                               'scheduled Analytics Rule could be created '
                               'specifically for that Anomaly Type, so an '
                               'alert and/or incident is generated everytime '
                               "that type of Anomaly happens.'\n",
                'tactics': [],
                'techniques': nan},
 'query': '// You can leave out Anomalies that are already monitored through '
          'other An

### KQL - Community Github Repos

In [15]:
# Read yaml config file
repos = read_config('repos.yaml')

#Compile list of github urls to download
repo_urls = []
for item in repos:
    url = get_repo_urls(item['Github']['repo'],item['Github']['branch'])
    repo_urls.append(url)

# Set output dir to current directory
output_dir = Path.cwd()

In [16]:
#download github urls one by one
for url in repo_urls:
    download_git_archive(url, output_dir)

Downloaded and Extracted Files successfully
Downloaded and Extracted Files successfully


In [18]:
src_path = str(Path.cwd())

parsed_kql_files_dict = []

for item in repos:
    repo_name = item['Github']['repo']
    branch_name = item['Github']['branch']
    list_of_dict = parse_kql_to_dict(repo_name, branch_name, src_path)
    for item in list_of_dict:
        parsed_kql_files_dict.append(item)
        
# display parsed sample record
pprint(parsed_kql_files_dict[1])

{'attributes': {},
 'query': '"//Query to find public IP addresses associated to AWS '
          'instances\\nAWSCloudTrail\\n| where EventName has '
          '\\"AllocateAddress\\"\\n| extend IPAssigned = '
          'tostring(parse_json(ResponseElements).publicIp)\\n| extend '
          'AllocationID = '
          'tostring(parse_json(ResponseElements).allocationId)\\n| project '
          'TimeGenerated, UserIdentityArn, UserIdentityAccountId, IPAssigned, '
          'AllocationID"',
 'query_name': 'AWS-PublicIPAddedtoInstance',
 'source_path': 'https://github.com/reprise99/Sentinel-Queries/tree/main/AWS-PublicIPAddedtoInstance.kql'}


### Markdown - Community Public Github Repos

In [39]:
src_path = str(Path.cwd())

parsed_md_files_dict = []

for item in repos:
    repo_name = item['Github']['repo']
    branch_name = item['Github']['branch']
    list_of_dict = parse_markdown_to_dict(repo_name, branch_name, src_path)
    for item in list_of_dict:
        parsed_md_files_dict.append(item)

# Display sample parsed kql query from markdown
parsed_md_files_dict[0]

Processing 6 markdonw files from repo: reprise99/Sentinel-Queries
Processing 3 markdonw files from repo: ugurkocde/KQL_Intune


{'query_name': 'n/a',
 'query': 'SigninLogs\n| where TimeGenerated > ago(14d)\n| where UserPrincipalName == "reprise_99@testdomain.com"\n| where ResultType == "0"\n| where AppDisplayName == "Microsoft Teams"\n| project TimeGenerated, Location, IPAddress, UserAgent\n```',
 'source_path': 'https://github.com/reprise99/Sentinel-Queries/tree/main/README.md'}

In [40]:
# merging records to create final dataset
merged_dict = sentinel_github_dict + parsed_kql_files_dict + parsed_md_files_dict

len(merged_dict)

2906

In [46]:
# output final dataset as json
with open("../data/kql_queries.json", "w") as final:
    json.dump(merged_dict, final)

In [42]:
df = pd.DataFrame.from_dict(merged_dict)
display(df.head())

,source_path,query_name,query,attributes
0,https://github.com/Azure/Azure-Sentinel/blob/m...,Unusual Anomaly,// You can leave out Anomalies that are alread...,{'description': ''Anomaly Rules generate event...
1,https://github.com/Azure/Azure-Sentinel/blob/m...,Brute force attack against user credentials (U...,let failureCountThreshold = 10;\nlet successCo...,{'description': ''Identifies evidence of brute...
2,https://github.com/Azure/Azure-Sentinel/blob/m...,Potential Password Spray Attack (Uses Authenti...,let FailureThreshold = 15;\nimAuthentication\n...,{'description': ''This query searches for fail...
3,https://github.com/Azure/Azure-Sentinel/blob/m...,User login from different countries within 3 h...,let timeframe = ago(3h);\nlet threshold = 2;\n...,{'description': ''This query searches for succ...
4,https://github.com/Azure/Azure-Sentinel/blob/m...,Sign-ins from IPs that attempt sign-ins to dis...,imAuthentication\n| where EventResult =='Failu...,{'description': ''Identifies IPs with failed a...


## Cleaning up downloaded files from github

In [43]:
!rm -rf 'Azure-Sentinel-master'

In [44]:
!rm -rf 'KQL_Intune-main'

In [45]:
!rm -rf 'Sentinel-Queries-main'